In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pickle
from sklearn.externals import joblib

C:\Users\bong\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
from sklearn.metrics import accuracy_score

# 데이터 불러오기

In [0]:
data= pd.read_csv("data5.csv",encoding = 'utf-8')

In [0]:
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,출발지연여부,지연여부,실제도착(시),실제출발(시),역간소요시간,요일1,환승가능,시간,승차,하차
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,False,False,5,5,0.0,1,2,5~6,1122.0,170.0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,False,False,5,5,2.0,1,0,5~6,152.0,43.0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,False,False,5,5,1.5,1,0,5~6,711.0,198.0
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,False,False,5,5,2.0,1,0,5~6,470.0,91.0
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,False,False,5,5,2.5,1,0,5~6,212.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,True,True,14,14,1.5,0,2,14~15,798.0,825.0
939747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,True,True,14,14,2.5,0,0,14~15,1887.0,2223.0
939748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,True,True,14,14,2.0,0,0,14~15,417.0,315.0
939749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,True,True,14,14,1.5,0,0,14~15,292.0,306.0


In [0]:
data['index'] = 0

# 인덱스 넣어주기 -> 표를 나눴다 붙일때 용이!

In [0]:
for i in range(0,data.shape[0]):
    data['index'].iat[i] = i

In [0]:
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,지연여부,실제도착(시),실제출발(시),역간소요시간,요일1,환승가능,시간,승차,하차,index
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,False,5,5,0.0,1,2,5~6,1122.0,170.0,0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,False,5,5,2.0,1,0,5~6,152.0,43.0,1
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,False,5,5,1.5,1,0,5~6,711.0,198.0,2
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,False,5,5,2.0,1,0,5~6,470.0,91.0,3
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,False,5,5,2.5,1,0,5~6,212.0,58.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,True,14,14,1.5,0,2,14~15,798.0,825.0,939746
939747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,True,14,14,2.5,0,0,14~15,1887.0,2223.0,939747
939748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,True,14,14,2.0,0,0,14~15,417.0,315.0,939748
939749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,True,14,14,1.5,0,0,14~15,292.0,306.0,939749


# 1시간전,  2시간전... 지연여부 컬럼

# 먼저 상행, 하행으로 나누어 지는지 확인하자!

In [0]:
data['출발행'].value_counts()

인천     198351
광운대     84696
용산      78396
청량리     70369
동인천     69393
소요산     68152
서동탄     64198
양주      51272
천안      50272
신창      48231
의정부     42316
동두천     36477
구로      34226
병점      15472
창동       9441
동묘앞      9168
부평       4206
서울       3315
영등포      1380
부천        420
Name: 출발행, dtype: int64

In [0]:
data['종착행'].value_counts()

인천     203025
광운대     82007
용산      80828
청량리     77231
동인천     69012
소요산     65773
서동탄     65090
양주      48643
천안      48545
신창      48207
의정부     40074
동두천     34394
구로      33365
병점      15582
동묘앞     12003
창동      11784
부평       2934
서울        864
부천        390
Name: 종착행, dtype: int64

# 데이터 출종착행 컬럼하나 만들고 유니크 찍어보기

# 출발-종착으로 이루어진 컬럼 

In [0]:
data['출종착행'] = data[['출발행','종착행']].apply(lambda x: x[0]+x[1],axis = 1)

In [0]:
len(data['출종착행'].unique())

97

In [0]:
data['출종착행'].unique()

array(['구로인천', '인천소요산', '부천용산', '동인천용산', '구로동인천', '용산동인천', '소요산인천',
       '의정부인천', '동인천구로', '인천의정부', '인천양주', '양주인천', '인천동두천', '동인천부천',
       '용산부평', '부평용산', '구로부평', '창동인천', '용산구로', '구로용산', '동두천인천', '동두천구로',
       '구로소요산', '용산신창', '용산천안', '신창용산', '천안용산', '용산병점', '병점용산', '부평동두천',
       '인천창동', '인천구로', '인천광운대', '영등포인천', '병점광운대', '구로서동탄', '광운대서동탄',
       '서동탄광운대', '서동탄청량리', '광운대병점', '서동탄구로', '청량리병점', '청량리서동탄', '병점청량리',
       '천안광운대', '천안신창', '천안청량리', '병점신창', '병점천안', '신창청량리', '구로신창', '신창광운대',
       '구로천안', '광운대천안', '신창병점', '청량리신창', '광운대신창', '청량리천안', '천안구로', '신창구로',
       '천안병점', '신창천안', '구로의정부', '양주구로', '소요산구로', '구로청량리', '구로광운대',
       '청량리구로', '구로동두천', '광운대인천', '광운대양주', '양주광운대', '창동동두천', '광운대소요산',
       '광운대동두천', '창동양주', '창동소요산', '소요산광운대', '광운대의정부', '동두천광운대', '의정부광운대',
       '동묘앞인천', '인천동묘앞', '인천청량리', '서울서동탄', '동묘앞서동탄', '서동탄동묘앞', '서울광운대',
       '서울양주', '청량리서울', '서울동묘앞', '창동구로', '광운대구로', '구로양주', '서동탄서울',
       '서울의정부', '광운대창동'], dtype=object)

# 직접 지도를 보고 상행하행 구분하기

# 상행  
구로인천  
인천소요산  
부천용산  
동인천용산  
동인천구로  
인천의정부  
인천양주  
인천동두천  
부평용산  
구로용산  
구로소요산  
신창용산  
천안용산  
용산병점  
병점용산  
부평동두천  
인천창동  
인천구로  
인천광운대  
병점광운대  
서동탄광운대  
서동탄청량리  
서동탄구로  
병점청량리  
천안광운대  
천안청량리  
신창청량리  
신창광운대  
신창병점  
천안구로  
신창구로  
천안병점  
신창천안  
구로의정부  
구로청량리  
구로광운대  
구로동두천  
광운대양주  
광운대소요산  
창동동두천  
광운대동두천  
창동소요산  
창동양주  
광운대의정부  
인천동묘앞  
인천청량리  
서울양주  
서동탄서울  
구로양주  
서동탄동묘앞  
서울광운대  
서울동묘앞  

# 하행
구로동인천  
용산동인천  
소요산인천  
의정부인천  
양주인천  
동인천부천  
용산부평  
구로부평  
창동인천  
용산구로  
동두천인천  
동두천구로  
용산신창  
용산천안  
용산병점  
영등포인천  
구로서동탄  
광운대서동탄  
광운대병점  
청량리병점  
병점신창  
천안신창  
구로신창  
병점천안  
청량리서동탄  
구로천안  
청량리신창  
양주구로  
소요산구로  
청량리천안  
광운대신창  
광운대천안  
청량리구로  
양주광운대  
광운대인천  
소요산광운대  
동두천광운대  
의정부광운대  
동묘앞인천  
서울서동탄  
청량리서울  
동묘앞서동탄  
창동구로  
광운대구로  

In [0]:
# 상행 -> up
up = [
'인천소요산',
'서울의정부',
'광운대창동',
'부천용산',
'동인천용산',
'동인천구로',
'인천의정부',
'인천양주',
'인천동두천',
'부평용산',
'구로용산',
'구로소요산',
'신창용산',
'천안용산',
'병점용산',
'부평동두천',
'인천창동',
'인천구로',
'인천광운대',
'병점광운대',
'서동탄광운대',
'서동탄청량리',
'서동탄구로',
'병점청량리',
'천안광운대',
'천안청량리',
'신창청량리',
'신창광운대',
'신창병점',
'천안구로',
'신창구로',
'천안병점',
'신창천안',
'구로의정부',
'구로청량리',
'구로광운대',
'구로동두천',
'광운대양주',
'광운대소요산',
'창동동두천',
'광운대동두천',
'창동소요산',
'창동양주',
'광운대의정부',
'인천동묘앞',
'인천청량리',
'서울양주',
'서동탄서울',
'구로양주',
'서동탄동묘앞',
'서울광운대',
'서울동묘앞']

# 상행이면 1 하행이면 0을 할당

In [0]:
data['상하행'] = data['출종착행'].apply(lambda x: 1 if x in up else 0)

In [0]:
len(data[data[['출종착행','상하행']]['상하행']==1]['출종착행'].value_counts())

52

In [0]:
len(data[data[['출종착행','상하행']]['상하행']==0]['출종착행'].value_counts())

45

# 5번째 전 역, 5번째 전 열차 칼럼

In [0]:
data['5번째전역_지연여부'] = False

In [0]:
data

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,역간소요시간,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,0.0,1,2,5~6,1122.0,170.0,0,구로인천,0,False
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,2.0,1,0,5~6,152.0,43.0,1,구로인천,0,False
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,1.5,1,0,5~6,711.0,198.0,2,구로인천,0,False
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,2.0,1,0,5~6,470.0,91.0,3,구로인천,0,False
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,2.5,1,0,5~6,212.0,58.0,4,구로인천,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,1.5,0,2,14~15,798.0,825.0,939746,인천양주,1,False
939747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,2.5,0,0,14~15,1887.0,2223.0,939747,인천양주,1,False
939748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,2.0,0,0,14~15,417.0,315.0,939748,인천양주,1,False
939749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,1.5,0,0,14~15,292.0,306.0,939749,인천양주,1,False


In [0]:
for i in range(0,data.shape[0]-5):
    if data['열차번호'].iat[i] ==  data['열차번호'].iat[i+5]:
        data['5번째전역_지연여부'].iat[i+5] = data['지연여부'].iat[i]

In [0]:
stations = list(data['역'].value_counts().index)

In [0]:
data['역정체여부'] = False

In [0]:
data[(data['상하행'] == 1) & (data['역']=='영등포')].sort_values(by='계획도착일시')

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부
27012,20191101,금,K802,영등포,정차,20191101050400,20191101050414,20191101050430,20191101050457,0,...,1,0,5~6,820.0,249.0,27012,구로의정부,1,False,False
27120,20191101,금,K806,영등포,정차,20191101051500,20191101051628,20191101051530,20191101051736,0,...,1,0,5~6,820.0,249.0,27120,구로소요산,1,False,False
27244,20191101,금,K810,영등포,정차,20191101052400,20191101052538,20191101052430,20191101052657,0,...,1,0,5~6,820.0,249.0,27244,구로의정부,1,False,False
11767,20191101,금,K2,영등포,정차,20191101053200,20191101053351,20191101053230,20191101053427,0,...,1,0,5~6,820.0,249.0,11767,부평동두천,1,True,False
27394,20191101,금,K818,영등포,정차,20191101054100,20191101054213,20191101054130,20191101054247,0,...,1,0,5~6,820.0,249.0,27394,구로소요산,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914952,20191130,토,K1148,영등포,정차,20191130232500,20191130232603,20191130232530,20191130232806,1,...,0,0,23~24,1993.0,1120.0,914952,동인천용산,1,False,False
937771,20191130,토,S238,영등포,정차,20191130233330,20191130233638,20191130233400,20191130233730,0,...,0,0,23~24,1993.0,1120.0,937771,인천동묘앞,1,True,False
939076,20191130,토,S518,영등포,정차,20191130234030,20191130234058,20191130234100,20191130234133,0,...,0,0,23~24,1993.0,1120.0,939076,서동탄동묘앞,1,False,False
922519,20191130,토,K240,영등포,정차,20191130235330,20191130235705,20191130235400,20191130235747,0,...,0,0,23~24,1993.0,1120.0,922519,인천광운대,1,True,False


# 동적변수생성 + 데이터프레임 집어넣기

In [0]:
import sys , numpy as np
mod = sys.modules[__name__]
for station in stations:
    setattr(mod, data[(data['상하행'] == 1) & (data['역']==station)].sort_values(by='계획도착일시')['역'].iat[0]+'1',  data[(data['상하행'] == 1) & (data['역']==station)].sort_values(by='계획도착일시'))

In [0]:
import sys , numpy as np
mod = sys.modules[__name__]
for station in stations:
    setattr(mod, data[(data['상하행'] == 0) & (data['역']==station)].sort_values(by='계획도착일시')['역'].iat[0]+'0',  data[(data['상하행'] == 0) & (data['역']==station)].sort_values(by='계획도착일시'))

In [0]:
stations_updw = []
for station in stations:
    stations_updw.append(station+'1')
    stations_updw.append(station+'0')

In [0]:
stations_updw

['구로1',
 '구로0',
 '영등포1',
 '영등포0',
 '신도림1',
 '신도림0',
 '노량진1',
 '노량진0',
 '용산1',
 '용산0',
 '대방1',
 '대방0',
 '신길1',
 '신길0',
 '서울1',
 '서울0',
 '동묘앞1',
 '동묘앞0',
 '동대문1',
 '동대문0',
 '종로5가1',
 '종로5가0',
 '종로3가1',
 '종로3가0',
 '종각1',
 '종각0',
 '시청1',
 '시청0',
 '남영1',
 '남영0',
 '제기동1',
 '제기동0',
 '청량리1',
 '청량리0',
 '신설동1',
 '신설동0',
 '부천1',
 '부천0',
 '송내1',
 '송내0',
 '구일1',
 '구일0',
 '오류동1',
 '오류동0',
 '온수1',
 '온수0',
 '역곡1',
 '역곡0',
 '부개1',
 '부개0',
 '중동1',
 '중동0',
 '부평1',
 '부평0',
 '개봉1',
 '개봉0',
 '소사1',
 '소사0',
 '도화1',
 '도화0',
 '제물포1',
 '제물포0',
 '백운1',
 '백운0',
 '주안1',
 '주안0',
 '동암1',
 '동암0',
 '동인천1',
 '동인천0',
 '도원1',
 '도원0',
 '간석1',
 '간석0',
 '광운대1',
 '광운대0',
 '석계1',
 '석계0',
 '회기1',
 '회기0',
 '신이문1',
 '신이문0',
 '외대앞1',
 '외대앞0',
 '병점1',
 '병점0',
 '수원1',
 '수원0',
 '세류1',
 '세류0',
 '군포1',
 '군포0',
 '안양1',
 '안양0',
 '금천구청1',
 '금천구청0',
 '의왕1',
 '의왕0',
 '관악1',
 '관악0',
 '가산디지털단지1',
 '가산디지털단지0',
 '독산1',
 '독산0',
 '석수1',
 '석수0',
 '창동1',
 '창동0',
 '월계1',
 '월계0',
 '녹천1',
 '녹천0',
 '인천1',
 '인천0',
 '방학1',
 '방학0',
 '의정부1',
 '의정부0',
 '도봉

In [0]:
영등포1

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부
27012,20191101,금,K802,영등포,정차,20191101050400,20191101050414,20191101050430,20191101050457,0,...,1,0,5~6,820.0,249.0,27012,구로의정부,1,False,False
27120,20191101,금,K806,영등포,정차,20191101051500,20191101051628,20191101051530,20191101051736,0,...,1,0,5~6,820.0,249.0,27120,구로소요산,1,False,False
27244,20191101,금,K810,영등포,정차,20191101052400,20191101052538,20191101052430,20191101052657,0,...,1,0,5~6,820.0,249.0,27244,구로의정부,1,False,False
11767,20191101,금,K2,영등포,정차,20191101053200,20191101053351,20191101053230,20191101053427,0,...,1,0,5~6,820.0,249.0,11767,부평동두천,1,True,False
27394,20191101,금,K818,영등포,정차,20191101054100,20191101054213,20191101054130,20191101054247,0,...,1,0,5~6,820.0,249.0,27394,구로소요산,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914952,20191130,토,K1148,영등포,정차,20191130232500,20191130232603,20191130232530,20191130232806,1,...,0,0,23~24,1993.0,1120.0,914952,동인천용산,1,False,False
937771,20191130,토,S238,영등포,정차,20191130233330,20191130233638,20191130233400,20191130233730,0,...,0,0,23~24,1993.0,1120.0,937771,인천동묘앞,1,True,False
939076,20191130,토,S518,영등포,정차,20191130234030,20191130234058,20191130234100,20191130234133,0,...,0,0,23~24,1993.0,1120.0,939076,서동탄동묘앞,1,False,False
922519,20191130,토,K240,영등포,정차,20191130235330,20191130235705,20191130235400,20191130235747,0,...,0,0,23~24,1993.0,1120.0,922519,인천광운대,1,True,False


In [0]:
stations = [구로1,
 구로0,
 영등포1,
 영등포0,
 신도림1,
 신도림0,
 용산1,
 용산0,
 노량진1,
 노량진0,
 신길1,
 신길0,
 대방1,
 대방0,
 서울1,
 서울0,
 종각1,
 종각0,
 시청1,
 시청0,
 종로3가1,
 종로3가0,
 종로5가1,
 종로5가0,
 동묘앞1,
 동묘앞0,
 동대문1,
 동대문0,
 남영1,
 남영0,
 청량리1,
 청량리0,
 신설동1,
 신설동0,
 제기동1,
 제기동0,
 부천1,
 부천0,
 구일1,
 구일0,
 오류동1,
 오류동0,
 송내1,
 송내0,
 부개1,
 부개0,
 소사1,
 소사0,
 개봉1,
 개봉0,
 중동1,
 중동0,
 부평1,
 부평0,
 역곡1,
 역곡0,
 온수1,
 온수0,
 백운1,
 백운0,
 도화1,
 도화0,
 동암1,
 동암0,
 도원1,
 도원0,
 동인천1,
 동인천0,
 간석1,
 간석0,
 제물포1,
 제물포0,
 주안1,
 주안0,
 광운대1,
 광운대0,
 신이문1,
 신이문0,
 석계1,
 석계0,
 외대앞1,
 외대앞0,
 회기1,
 회기0,
 병점1,
 병점0,
 수원1,
 수원0,
 군포1,
 군포0,
 세류1,
 세류0,
 안양1,
 안양0,
 금천구청1,
 금천구청0,
 의왕1,
 의왕0,
 독산1,
 독산0,
 가산디지털단지1,
 가산디지털단지0,
 관악1,
 관악0,
 석수1,
 석수0,
 창동1,
 창동0,
 녹천1,
 녹천0,
 월계1,
 월계0,
 인천1,
 인천0,
 도봉산1,
 도봉산0,
 의정부1,
 의정부0,
 회룡1,
 회룡0,
 방학1,
 방학0,
 망월사1,
 망월사0,
 도봉1,
 도봉0,
 당정1,
 당정0,
 화서1,
 화서0,
 금정1,
 금정0,
 명학1,
 명학0,
 성균관대1,
 성균관대0,
 가능1,
 가능0,
 녹양1,
 녹양0,
 양주1,
 양주0,
 천안1,
 천안0,
 서정리1,
 서정리0,
 두정1,
 두정0,
 오산1,
 오산0,
 직산1,
 직산0,
 송탄1,
 송탄0,
 진위1,
 진위0,
 성환1,
 성환0,
 오산대1,
 오산대0,
 세마1,
 세마0,
 평택1,
 평택0,
 지제1,
 지제0,
 덕계1,
 덕계0,
 덕정1,
 덕정0,
 지행1,
 지행0,
 동두천1,
 동두천0,
 보산1,
 보산0,
 동두천중앙1,
 동두천중앙0,
 서동탄1,
 서동탄0,
 소요산1,
 소요산0,
 배방1,
 배방0,
 쌍용1,
 쌍용0,
 봉명1,
 봉명0,
 아산1,
 아산0,
 신창1,
 신창0,
 온양온천1,
 온양온천0]

In [0]:
for station in stations:
    for i in range(0,len(station)-5):
        station['역정체여부'].iat[i+5] = station['지연여부'].iat[i]

# 지연여부, 역정체여부가 잘 들어 갔는지 확인

In [0]:
서울1[['계획도착일시','지연여부','역정체여부']][:60]

,계획도착일시,지연여부,역정체여부
27018,20191101051930,True,False
32376,20191101052700,False,False
27126,20191101053030,True,False
32391,20191101053500,False,False
27250,20191101053930,True,False
11773,20191101054730,True,True
32418,20191101055300,False,False
27400,20191101055630,False,True
32433,20191101060300,False,False
15613,20191101060700,False,True


In [0]:
data['역정체여부'].value_counts()

False    939751
Name: 역정체여부, dtype: int64

In [0]:
data_full = pd.DataFrame()
for station in stations:
     data_full = data_full.append(station)
    

In [0]:
data = data_full.sort_values(by='index')

# 트레인 테스트 나누고 지연 통합 컬럼생성

In [0]:
X_train = data.iloc[:730000,:]
X_test = data.iloc[730000:,:]

In [0]:
X_train['지연 통합'] = X_train[['도착지연(분)','도착지연(초)']].apply(lambda x: (x[0]*60)+(x[1]),axis = 1)

C:\Users\bong\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
X_train

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,지연 통합
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,2,5~6,1122.0,170.0,0,구로인천,0,False,False,0.0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,0,5~6,152.0,43.0,1,구로인천,0,False,False,7.0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,0,5~6,711.0,198.0,2,구로인천,0,False,False,9.0
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,0,5~6,470.0,91.0,3,구로인천,0,False,False,0.0
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,0,5~6,212.0,58.0,4,구로인천,0,False,False,-28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729995,20191124,일,K196,동인천,정차,20191124195130,20191124194759,20191124195200,20191124195639,0,...,0,19~20,752.0,585.0,729995,인천의정부,1,False,False,-211.0
729996,20191124,일,K196,도원,정차,20191124195330,20191124195905,20191124195400,20191124195933,0,...,0,19~20,58.0,126.0,729996,인천의정부,1,False,False,335.0
729997,20191124,일,K196,제물포,정차,20191124195530,20191124200115,20191124195600,20191124200135,0,...,0,20~21,170.0,425.0,729997,인천의정부,1,False,False,345.0
729998,20191124,일,K196,도화,정차,20191124195730,20191124200254,20191124195800,20191124200326,0,...,0,20~21,50.0,143.0,729998,인천의정부,1,False,False,324.0


In [0]:
X_train['실지연시간'] = 0

C:\Users\bong\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
X_train['지연 통합']

0           0.0
1           7.0
2           9.0
3           0.0
4         -28.0
          ...  
729995   -211.0
729996    335.0
729997    345.0
729998    324.0
729999    309.0
Name: 지연 통합, Length: 730000, dtype: float64

# 열차가 달린 한구간이 지연에 실제로 얼마나 기여했는지

In [0]:
for i in range(0, X_train.shape[0]-1):
    if X_train['열차번호'].iat[i] == X_train['열차번호'].iat[i+1]:

        X_train['실지연시간'].iat[i+1] = X_train['지연 통합'].iat[i+1] - X_train['지연 통합'].iat[i]

    else:
        X_train['실지연시간'].iat[i+1] = X_train['지연 통합'].iat[i+1]
       
    

# 60초 넘으면 지연으로 판단 -> 실 지연여부

In [0]:
X_train['실지연여부'] = X_train['실지연시간'].apply(lambda x: 1 if x>=60 else 0)

C:\Users\bong\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# 실지연여부를 카운트 해서 가중치로만듬

In [0]:
temp = pd.DataFrame(X_train.groupby(['역','시간'])['실지연여부'].sum())
temp.iloc[:60,:]

실지연여부
역       시간          
가능      0~1        0
        10~11      8
        11~12      3
        12~13      5
        13~14      7
        14~15      5
        15~16      3
        16~17      1
        17~18      7
        18~19      2
        19~20      4
        20~21      3
        21~22      3
        22~23      1
        23~24      2
        5~6        0
        6~7       14
        7~8        9
        8~9       25
        9~10       6
가산디지털단지 0~1        0
        10~11     37
        11~12     21
        12~13     14
        13~14     16
        14~15      9
        15~16     20
        16~17     21
        17~18     24
        18~19     34
        19~20     73
        20~21     31
        21~22      8
        22~23     22
        23~24      1
        5~6       13
        6~7       35
        7~8        7
        8~9       29
        9~10      46
간석      0~1        0
        10~11     37
        11~12     20
        12~13     23
        13~14     27
        14~15     33
        15~16     36
        16~17     20
        17~18     30
        18~19     44
        19~20     28
        20~21     30
        21~22     30
        22~23     22
        23~24     13
        5~6       10
        6~7       15
        7~8       27
        8~9       38
        9~10      31

In [0]:
temp.columns=['역_시간_실지연가중치']

In [0]:
temp1 = pd.DataFrame(X_train.groupby(['운행일자','역','시간'])['실지연여부'].sum())

In [0]:
X_train_1 = X_train.merge(temp, on=['시간','역'],how='left')

In [0]:
X_test_1 = X_test.merge(temp, on=['시간','역'],how='left')

In [0]:
X_train_1

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,지연 통합,실지연시간,실지연여부,역_시간_실지연가중치
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,170.0,0,구로인천,0,False,False,0.0,0,0,12.0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,43.0,1,구로인천,0,False,False,7.0,7,0,40.0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,198.0,2,구로인천,0,False,False,9.0,2,0,1.0
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,91.0,3,구로인천,0,False,False,0.0,-9,0,0.0
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,58.0,4,구로인천,0,False,False,-28.0,-28,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729995,20191124,일,K196,동인천,정차,20191124195130,20191124194759,20191124195200,20191124195639,0,...,585.0,729995,인천의정부,1,False,False,-211.0,-211,0,104.0
729996,20191124,일,K196,도원,정차,20191124195330,20191124195905,20191124195400,20191124195933,0,...,126.0,729996,인천의정부,1,False,False,335.0,546,1,158.0
729997,20191124,일,K196,제물포,정차,20191124195530,20191124200115,20191124195600,20191124200135,0,...,425.0,729997,인천의정부,1,False,False,345.0,10,0,2.0
729998,20191124,일,K196,도화,정차,20191124195730,20191124200254,20191124195800,20191124200326,0,...,143.0,729998,인천의정부,1,False,False,324.0,-21,0,6.0


In [0]:
temp[list(map(lambda x: True if (x[0] == '도화') else False, temp.index))]

역_시간_실지연가중치
역  시간                
도화 0~1              2
   10~11            4
   11~12            0
   12~13            1
   13~14            4
   14~15            2
   15~16            2
   16~17            3
   17~18            5
   18~19            8
   19~20           11
   20~21            6
   21~22            8
   22~23            8
   23~24            6
   5~6              2
   6~7             11
   7~8             11
   8~9              5
   9~10             4

In [0]:
X_test_1

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치
0,20191124,일,K196,간석,정차,20191124200130,20191124200705,20191124200200,20191124200727,0,...,0,20~21,110.0,223.0,730000,인천의정부,1,False,False,30.0
1,20191124,일,K196,동암,정차,20191124200400,20191124200901,20191124200430,20191124200926,0,...,0,20~21,371.0,690.0,730001,인천의정부,1,True,False,0.0
2,20191124,일,K196,백운,정차,20191124200630,20191124201115,20191124200700,20191124201142,0,...,0,20~21,158.0,289.0,730002,인천의정부,1,True,False,4.0
3,20191124,일,K196,부평,정차,20191124200900,20191124201356,20191124200930,20191124201555,0,...,0,20~21,1596.0,1365.0,730003,인천의정부,1,True,False,23.0
4,20191124,일,K196,부개,정차,20191124201130,20191124201739,20191124201200,20191124201807,0,...,0,20~21,153.0,388.0,730004,인천의정부,1,True,False,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,2,14~15,798.0,825.0,939746,인천양주,1,True,True,17.0
209747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,0,14~15,1887.0,2223.0,939747,인천양주,1,True,True,10.0
209748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,0,14~15,417.0,315.0,939748,인천양주,1,True,True,5.0
209749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,0,14~15,292.0,306.0,939749,인천양주,1,True,True,22.0


In [0]:
X_test_1['일시_merge'] = X_test_1['계획도착일시'].apply(lambda x: str(x)[6:10])

In [0]:
X_test_1

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,20191124,일,K196,간석,정차,20191124200130,20191124200705,20191124200200,20191124200727,0,...,20~21,110.0,223.0,730000,인천의정부,1,False,False,30.0,2420
1,20191124,일,K196,동암,정차,20191124200400,20191124200901,20191124200430,20191124200926,0,...,20~21,371.0,690.0,730001,인천의정부,1,True,False,0.0,2420
2,20191124,일,K196,백운,정차,20191124200630,20191124201115,20191124200700,20191124201142,0,...,20~21,158.0,289.0,730002,인천의정부,1,True,False,4.0,2420
3,20191124,일,K196,부평,정차,20191124200900,20191124201356,20191124200930,20191124201555,0,...,20~21,1596.0,1365.0,730003,인천의정부,1,True,False,23.0,2420
4,20191124,일,K196,부개,정차,20191124201130,20191124201739,20191124201200,20191124201807,0,...,20~21,153.0,388.0,730004,인천의정부,1,True,False,19.0,2420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,14~15,798.0,825.0,939746,인천양주,1,True,True,17.0,3014
209747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,14~15,1887.0,2223.0,939747,인천양주,1,True,True,10.0,3014
209748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,14~15,417.0,315.0,939748,인천양주,1,True,True,5.0,3014
209749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,14~15,292.0,306.0,939749,인천양주,1,True,True,22.0,3014


In [0]:
temp[list(map(lambda x: True if (x[0] == '간석') else False, temp.index))]

역_시간_실지연가중치
역  시간                
간석 0~1              0
   10~11           37
   11~12           20
   12~13           23
   13~14           27
   14~15           33
   15~16           36
   16~17           20
   17~18           30
   18~19           44
   19~20           28
   20~21           30
   21~22           30
   22~23           22
   23~24           13
   5~6             10
   6~7             15
   7~8             27
   8~9             38
   9~10            31

In [0]:
X_train_1

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,지연 통합,실지연시간,실지연여부,역_시간_실지연가중치
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,170.0,0,구로인천,0,False,False,0.0,0,0,12.0
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,43.0,1,구로인천,0,False,False,7.0,7,0,40.0
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,198.0,2,구로인천,0,False,False,9.0,2,0,1.0
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,91.0,3,구로인천,0,False,False,0.0,-9,0,0.0
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,58.0,4,구로인천,0,False,False,-28.0,-28,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729995,20191124,일,K196,동인천,정차,20191124195130,20191124194759,20191124195200,20191124195639,0,...,585.0,729995,인천의정부,1,False,False,-211.0,-211,0,104.0
729996,20191124,일,K196,도원,정차,20191124195330,20191124195905,20191124195400,20191124195933,0,...,126.0,729996,인천의정부,1,False,False,335.0,546,1,158.0
729997,20191124,일,K196,제물포,정차,20191124195530,20191124200115,20191124195600,20191124200135,0,...,425.0,729997,인천의정부,1,False,False,345.0,10,0,2.0
729998,20191124,일,K196,도화,정차,20191124195730,20191124200254,20191124195800,20191124200326,0,...,143.0,729998,인천의정부,1,False,False,324.0,-21,0,6.0


In [0]:
X_train_1['일시_merge'] = X_train_1['계획도착일시'].apply(lambda x: str(x)[6:10])

In [0]:
X_train_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730000 entries, 0 to 729999
Data columns (total 37 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   운행일자         730000 non-null  int64  
 1   요일           730000 non-null  object 
 2   열차번호         730000 non-null  object 
 3   역            730000 non-null  object 
 4   출도착구분        730000 non-null  object 
 5   계획도착일시       730000 non-null  int64  
 6   실제도착일시       730000 non-null  object 
 7   계획출발일시       730000 non-null  int64  
 8   실제출발일시       730000 non-null  object 
 9   급행여부         730000 non-null  int64  
 10  출발행          730000 non-null  object 
 11  종착행          730000 non-null  object 
 12  도착지연(분)      730000 non-null  float64
 13  도착지연(초)      730000 non-null  float64
 14  출발지연(분)      730000 non-null  float64
 15  출발지연(초)      730000 non-null  float64
 16  도착지연여부       730000 non-null  bool   
 17  출발지연여부       730000 non-null  bool   
 18  지연여부         730000 non-

In [0]:
data_total = X_train_1

In [0]:
data_total

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,index,출종착행,상하행,5번째전역_지연여부,역정체여부,지연 통합,실지연시간,실지연여부,역_시간_실지연가중치,일시_merge
0,20191101,금,K1,구로,시발,20191101050000,,20191101050000,20191101050015,0,...,0,구로인천,0,False,False,0.0,0,0,12.0,0105
1,20191101,금,K1,구일,정차,20191101050200,20191101050207,20191101050230,20191101050247,0,...,1,구로인천,0,False,False,7.0,7,0,40.0,0105
2,20191101,금,K1,개봉,정차,20191101050400,20191101050409,20191101050430,20191101050441,0,...,2,구로인천,0,False,False,9.0,2,0,1.0,0105
3,20191101,금,K1,오류동,정차,20191101050630,20191101050630,20191101050700,20191101050700,0,...,3,구로인천,0,False,False,0.0,-9,0,0.0,0105
4,20191101,금,K1,온수,정차,20191101050930,20191101050902,20191101051000,20191101051035,0,...,4,구로인천,0,False,False,-28.0,-28,0,1.0,0105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729995,20191124,일,K196,동인천,정차,20191124195130,20191124194759,20191124195200,20191124195639,0,...,729995,인천의정부,1,False,False,-211.0,-211,0,104.0,2419
729996,20191124,일,K196,도원,정차,20191124195330,20191124195905,20191124195400,20191124195933,0,...,729996,인천의정부,1,False,False,335.0,546,1,158.0,2419
729997,20191124,일,K196,제물포,정차,20191124195530,20191124200115,20191124195600,20191124200135,0,...,729997,인천의정부,1,False,False,345.0,10,0,2.0,2419
729998,20191124,일,K196,도화,정차,20191124195730,20191124200254,20191124195800,20191124200326,0,...,729998,인천의정부,1,False,False,324.0,-21,0,6.0,2419


In [0]:
X_test_1

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,20191124,일,K196,간석,정차,20191124200130,20191124200705,20191124200200,20191124200727,0,...,20~21,110.0,223.0,730000,인천의정부,1,False,False,30.0,2420
1,20191124,일,K196,동암,정차,20191124200400,20191124200901,20191124200430,20191124200926,0,...,20~21,371.0,690.0,730001,인천의정부,1,True,False,0.0,2420
2,20191124,일,K196,백운,정차,20191124200630,20191124201115,20191124200700,20191124201142,0,...,20~21,158.0,289.0,730002,인천의정부,1,True,False,4.0,2420
3,20191124,일,K196,부평,정차,20191124200900,20191124201356,20191124200930,20191124201555,0,...,20~21,1596.0,1365.0,730003,인천의정부,1,True,False,23.0,2420
4,20191124,일,K196,부개,정차,20191124201130,20191124201739,20191124201200,20191124201807,0,...,20~21,153.0,388.0,730004,인천의정부,1,True,False,19.0,2420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,14~15,798.0,825.0,939746,인천양주,1,True,True,17.0,3014
209747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,14~15,1887.0,2223.0,939747,인천양주,1,True,True,10.0,3014
209748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,14~15,417.0,315.0,939748,인천양주,1,True,True,5.0,3014
209749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,14~15,292.0,306.0,939749,인천양주,1,True,True,22.0,3014


In [0]:
data_total_test = X_test_1

In [0]:
data_total_test

,운행일자,요일,열차번호,역,출도착구분,계획도착일시,실제도착일시,계획출발일시,실제출발일시,급행여부,...,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,20191124,일,K196,간석,정차,20191124200130,20191124200705,20191124200200,20191124200727,0,...,20~21,110.0,223.0,730000,인천의정부,1,False,False,30.0,2420
1,20191124,일,K196,동암,정차,20191124200400,20191124200901,20191124200430,20191124200926,0,...,20~21,371.0,690.0,730001,인천의정부,1,True,False,0.0,2420
2,20191124,일,K196,백운,정차,20191124200630,20191124201115,20191124200700,20191124201142,0,...,20~21,158.0,289.0,730002,인천의정부,1,True,False,4.0,2420
3,20191124,일,K196,부평,정차,20191124200900,20191124201356,20191124200930,20191124201555,0,...,20~21,1596.0,1365.0,730003,인천의정부,1,True,False,23.0,2420
4,20191124,일,K196,부개,정차,20191124201130,20191124201739,20191124201200,20191124201807,0,...,20~21,153.0,388.0,730004,인천의정부,1,True,False,19.0,2420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209746,20191130,토,S98,회룡,정차,20191130142500,20191130142804,20191130142530,20191130142848,0,...,14~15,798.0,825.0,939746,인천양주,1,True,True,17.0,3014
209747,20191130,토,S98,의정부,정차,20191130142800,20191130143059,20191130142830,20191130143120,0,...,14~15,1887.0,2223.0,939747,인천양주,1,True,True,10.0,3014
209748,20191130,토,S98,가능,정차,20191130143030,20191130143301,20191130143100,20191130143341,0,...,14~15,417.0,315.0,939748,인천양주,1,True,True,5.0,3014
209749,20191130,토,S98,녹양,정차,20191130143230,20191130143504,20191130143300,20191130143620,0,...,14~15,292.0,306.0,939749,인천양주,1,True,True,22.0,3014


In [0]:
 data_total.drop(['출발행','종착행','지연 통합','운행일자','계획도착일시','도착지연여부','출발지연여부','요일','실제도착일시','계획출발일시','실제출발일시','도착지연(분)', '도착지연(초)', '출발지연(분)','출발지연(초)','도착지연여부', '출발지연여부','실제도착(시)', '실제출발(시)','역간소요시간','실지연시간','실지연여부'], inplace = True, axis =1)

In [0]:
 data_total

,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,K1,구로,시발,0,False,1,2,5~6,1122.0,170.0,0,구로인천,0,False,False,12.0,0105
1,K1,구일,정차,0,False,1,0,5~6,152.0,43.0,1,구로인천,0,False,False,40.0,0105
2,K1,개봉,정차,0,False,1,0,5~6,711.0,198.0,2,구로인천,0,False,False,1.0,0105
3,K1,오류동,정차,0,False,1,0,5~6,470.0,91.0,3,구로인천,0,False,False,0.0,0105
4,K1,온수,정차,0,False,1,0,5~6,212.0,58.0,4,구로인천,0,False,False,1.0,0105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729995,K196,동인천,정차,0,False,0,0,19~20,752.0,585.0,729995,인천의정부,1,False,False,104.0,2419
729996,K196,도원,정차,0,True,0,0,19~20,58.0,126.0,729996,인천의정부,1,False,False,158.0,2419
729997,K196,제물포,정차,0,True,0,0,20~21,170.0,425.0,729997,인천의정부,1,False,False,2.0,2419
729998,K196,도화,정차,0,True,0,0,20~21,50.0,143.0,729998,인천의정부,1,False,False,6.0,2419


In [0]:
 data_total_test.drop(['출발행','종착행','운행일자','계획도착일시','도착지연여부','출발지연여부','요일','실제도착일시','계획출발일시','실제출발일시','도착지연(분)', '도착지연(초)', '출발지연(분)','출발지연(초)','도착지연여부', '출발지연여부','실제도착(시)', '실제출발(시)','역간소요시간'], inplace = True, axis =1)

In [0]:
 data_total_test

,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,K196,간석,정차,0,True,0,0,20~21,110.0,223.0,730000,인천의정부,1,False,False,30.0,2420
1,K196,동암,정차,0,True,0,0,20~21,371.0,690.0,730001,인천의정부,1,True,False,0.0,2420
2,K196,백운,정차,0,True,0,0,20~21,158.0,289.0,730002,인천의정부,1,True,False,4.0,2420
3,K196,부평,정차,0,True,0,0,20~21,1596.0,1365.0,730003,인천의정부,1,True,False,23.0,2420
4,K196,부개,정차,0,True,0,0,20~21,153.0,388.0,730004,인천의정부,1,True,False,19.0,2420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209746,S98,회룡,정차,0,True,0,2,14~15,798.0,825.0,939746,인천양주,1,True,True,17.0,3014
209747,S98,의정부,정차,0,True,0,0,14~15,1887.0,2223.0,939747,인천양주,1,True,True,10.0,3014
209748,S98,가능,정차,0,True,0,0,14~15,417.0,315.0,939748,인천양주,1,True,True,5.0,3014
209749,S98,녹양,정차,0,True,0,0,14~15,292.0,306.0,939749,인천양주,1,True,True,22.0,3014


# 시간 칼럼 결측치 제거

In [0]:
data_total[data_total['시간'].isnull()]

,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
50,K10,시청,정차,0,False,1,2,NaN,NaN,NaN,50,인천소요산,1,True,True,NaN,0106
51,K10,종각,정차,0,True,1,0,NaN,NaN,NaN,51,인천소요산,1,True,True,NaN,0106
52,K10,종로3가,정차,0,True,1,3,NaN,NaN,NaN,52,인천소요산,1,True,True,NaN,0107
53,K10,종로5가,정차,0,True,1,0,NaN,NaN,NaN,53,인천소요산,1,False,True,NaN,0107
54,K10,동대문,정차,0,True,1,2,NaN,NaN,NaN,54,인천소요산,1,True,True,NaN,0107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729836,K195,종로3가,정차,0,True,0,3,NaN,NaN,NaN,729836,소요산인천,0,True,True,NaN,2419
729837,K195,종각,정차,0,True,0,0,NaN,NaN,NaN,729837,소요산인천,0,True,True,NaN,2419
729838,K195,시청,정차,0,True,0,2,NaN,NaN,NaN,729838,소요산인천,0,True,True,NaN,2419
729959,K1953,쌍용,정차,1,True,0,0,NaN,NaN,NaN,729959,용산신창,0,True,True,NaN,2415


In [0]:
data_total = data_total.drop(data_total[data_total['시간'].isnull()].index)

In [0]:
data_total = data_total.reset_index()

In [0]:
data_total.drop('index',axis = 1, inplace = True)

In [0]:
data_total

,level_0,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,0,K1,구로,시발,0,False,1,2,5~6,1122.0,170.0,구로인천,0,False,False,12.0,0105
1,1,K1,구일,정차,0,False,1,0,5~6,152.0,43.0,구로인천,0,False,False,40.0,0105
2,2,K1,개봉,정차,0,False,1,0,5~6,711.0,198.0,구로인천,0,False,False,1.0,0105
3,3,K1,오류동,정차,0,False,1,0,5~6,470.0,91.0,구로인천,0,False,False,0.0,0105
4,4,K1,온수,정차,0,False,1,0,5~6,212.0,58.0,구로인천,0,False,False,1.0,0105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628450,729995,K196,동인천,정차,0,False,0,0,19~20,752.0,585.0,인천의정부,1,False,False,104.0,2419
628451,729996,K196,도원,정차,0,True,0,0,19~20,58.0,126.0,인천의정부,1,False,False,158.0,2419
628452,729997,K196,제물포,정차,0,True,0,0,20~21,170.0,425.0,인천의정부,1,False,False,2.0,2419
628453,729998,K196,도화,정차,0,True,0,0,20~21,50.0,143.0,인천의정부,1,False,False,6.0,2419


In [0]:
 data_total_test

,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,index,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,K196,간석,정차,0,True,0,0,20~21,110.0,223.0,730000,인천의정부,1,False,False,30.0,2420
1,K196,동암,정차,0,True,0,0,20~21,371.0,690.0,730001,인천의정부,1,True,False,0.0,2420
2,K196,백운,정차,0,True,0,0,20~21,158.0,289.0,730002,인천의정부,1,True,False,4.0,2420
3,K196,부평,정차,0,True,0,0,20~21,1596.0,1365.0,730003,인천의정부,1,True,False,23.0,2420
4,K196,부개,정차,0,True,0,0,20~21,153.0,388.0,730004,인천의정부,1,True,False,19.0,2420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209746,S98,회룡,정차,0,True,0,2,14~15,798.0,825.0,939746,인천양주,1,True,True,17.0,3014
209747,S98,의정부,정차,0,True,0,0,14~15,1887.0,2223.0,939747,인천양주,1,True,True,10.0,3014
209748,S98,가능,정차,0,True,0,0,14~15,417.0,315.0,939748,인천양주,1,True,True,5.0,3014
209749,S98,녹양,정차,0,True,0,0,14~15,292.0,306.0,939749,인천양주,1,True,True,22.0,3014


In [0]:
data_total_test = data_total_test.drop(data_total_test[data_total_test['시간'].isnull()].index)

In [0]:
data_total_test = data_total_test.reset_index() 

In [0]:
data_total_test.drop('index',axis = 1, inplace = True)

In [0]:
data_total_test

,level_0,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,0,K196,간석,정차,0,True,0,0,20~21,110.0,223.0,인천의정부,1,False,False,30.0,2420
1,1,K196,동암,정차,0,True,0,0,20~21,371.0,690.0,인천의정부,1,True,False,0.0,2420
2,2,K196,백운,정차,0,True,0,0,20~21,158.0,289.0,인천의정부,1,True,False,4.0,2420
3,3,K196,부평,정차,0,True,0,0,20~21,1596.0,1365.0,인천의정부,1,True,False,23.0,2420
4,4,K196,부개,정차,0,True,0,0,20~21,153.0,388.0,인천의정부,1,True,False,19.0,2420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179819,209746,S98,회룡,정차,0,True,0,2,14~15,798.0,825.0,인천양주,1,True,True,17.0,3014
179820,209747,S98,의정부,정차,0,True,0,0,14~15,1887.0,2223.0,인천양주,1,True,True,10.0,3014
179821,209748,S98,가능,정차,0,True,0,0,14~15,417.0,315.0,인천양주,1,True,True,5.0,3014
179822,209749,S98,녹양,정차,0,True,0,0,14~15,292.0,306.0,인천양주,1,True,True,22.0,3014


# 데이터 프레임 합쳐서 인코딩하기
### 1) 합치기

In [0]:
 _data = data_total.append( data_total_test)

In [0]:
 _data

,level_0,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,출종착행,상하행,5번째전역_지연여부,역정체여부,역_시간_실지연가중치,일시_merge
0,0,K1,구로,시발,0,False,1,2,5~6,1122.0,170.0,구로인천,0,False,False,12.0,0105
1,1,K1,구일,정차,0,False,1,0,5~6,152.0,43.0,구로인천,0,False,False,40.0,0105
2,2,K1,개봉,정차,0,False,1,0,5~6,711.0,198.0,구로인천,0,False,False,1.0,0105
3,3,K1,오류동,정차,0,False,1,0,5~6,470.0,91.0,구로인천,0,False,False,0.0,0105
4,4,K1,온수,정차,0,False,1,0,5~6,212.0,58.0,구로인천,0,False,False,1.0,0105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179819,209746,S98,회룡,정차,0,True,0,2,14~15,798.0,825.0,인천양주,1,True,True,17.0,3014
179820,209747,S98,의정부,정차,0,True,0,0,14~15,1887.0,2223.0,인천양주,1,True,True,10.0,3014
179821,209748,S98,가능,정차,0,True,0,0,14~15,417.0,315.0,인천양주,1,True,True,5.0,3014
179822,209749,S98,녹양,정차,0,True,0,0,14~15,292.0,306.0,인천양주,1,True,True,22.0,3014


### 2) 필요없는 칼럼 지우기

In [0]:
_data = _data.drop(['일시_merge','상하행', '출종착행'],axis = 1)

In [0]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808279 entries, 0 to 179823
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   level_0      808279 non-null  int64  
 1   열차번호         808279 non-null  object 
 2   역            808279 non-null  object 
 3   출도착구분        808279 non-null  object 
 4   급행여부         808279 non-null  int64  
 5   지연여부         808279 non-null  bool   
 6   요일1          808279 non-null  int64  
 7   환승가능         808279 non-null  int64  
 8   시간           808279 non-null  object 
 9   승차           808279 non-null  float64
 10  하차           808279 non-null  float64
 11  5번째전역_지연여부   808279 non-null  bool   
 12  역정체여부        808279 non-null  bool   
 13  역_시간_실지연가중치  808279 non-null  float64
dtypes: bool(3), float64(3), int64(4), object(4)
memory usage: 76.3+ MB


# 인코딩

In [0]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808279 entries, 0 to 179823
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   level_0      808279 non-null  int64  
 1   열차번호         808279 non-null  object 
 2   역            808279 non-null  object 
 3   출도착구분        808279 non-null  object 
 4   급행여부         808279 non-null  int64  
 5   지연여부         808279 non-null  bool   
 6   요일1          808279 non-null  int64  
 7   환승가능         808279 non-null  int64  
 8   시간           808279 non-null  object 
 9   승차           808279 non-null  float64
 10  하차           808279 non-null  float64
 11  5번째전역_지연여부   808279 non-null  bool   
 12  역정체여부        808279 non-null  bool   
 13  역_시간_실지연가중치  808279 non-null  float64
dtypes: bool(3), float64(3), int64(4), object(4)
memory usage: 76.3+ MB


In [0]:
from sklearn import preprocessing

In [0]:
features = ['역', '출도착구분', '급행여부','열차번호', '지연여부', '요일1', '환승가능', '시간',
       '역_시간_실지연가중치','5번째전역_지연여부','역정체여부']
for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(_data.loc[:,feature])
        _data.loc[:,feature]=le.transform(_data.loc[:,feature])


In [0]:
_data.drop('level_0',inplace=True,axis=1)

In [0]:
_data

,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,5번째전역_지연여부,역정체여부,역_시간_실지연가중치
0,0,6,0,0,0,1,1,16,1122.0,170.0,0,0,12
1,0,7,1,0,0,1,0,16,152.0,43.0,0,0,40
2,0,3,1,0,0,1,0,16,711.0,198.0,0,0,1
3,0,62,1,0,0,1,0,16,470.0,91.0,0,0,0
4,0,65,1,0,0,1,0,16,212.0,58.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179819,1235,85,1,0,1,0,1,5,798.0,825.0,1,1,17
179820,1235,71,1,0,1,0,0,5,1887.0,2223.0,1,1,10
179821,1235,0,1,0,1,0,0,5,417.0,315.0,1,1,5
179822,1235,13,1,0,1,0,0,5,292.0,306.0,1,1,22


In [0]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808279 entries, 0 to 179823
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   열차번호         808279 non-null  int32  
 1   역            808279 non-null  int32  
 2   출도착구분        808279 non-null  int32  
 3   급행여부         808279 non-null  int64  
 4   지연여부         808279 non-null  int64  
 5   요일1          808279 non-null  int64  
 6   환승가능         808279 non-null  int64  
 7   시간           808279 non-null  int32  
 8   승차           808279 non-null  float64
 9   하차           808279 non-null  float64
 10  5번째전역_지연여부   808279 non-null  int64  
 11  역정체여부        808279 non-null  int64  
 12  역_시간_실지연가중치  808279 non-null  int64  
dtypes: float64(2), int32(4), int64(7)
memory usage: 74.0 MB


In [0]:
_data.to_csv('_data.csv')